In a circuit, we need to do three jobs: 

1. Encode the input
2. Do computation
3. Extract an output

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3,3) # create a quantum circuit with 3 qubits and 3 outputs
qc.x([0,1]) # apply X (NOT) gate on qubits 0 & 1
qc.measure([0,1,2],[0,1,2]) # measure qubits 0, 1 & 2 to classical bits 0, 1 & 2 respectively
qc.draw() # draw the circuit

┌───┐┌─┐   
q_0: ┤ X ├┤M├───
     ├───┤└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └┬─┬┘ ║ └╥┘
q_2: ─┤M├──╫──╫─
      └╥┘  ║  ║ 
c: 3/══╩═══╩══╩═
       2   0  1

In [2]:
from qiskit_aer import AerSimulator
sim = AerSimulator() # make new simulator object
job = sim.run(qc) # run the experiment with quantum circuit called qc
result = job.result() # get the results
result.get_counts() # interpret results as a "counts" dictionary

{'011': 1024}

In [3]:
qc = QuantumCircuit(3, 3)
qc.x(2)       # Fix q2=1 -> pattern 1__  (this is the MSB in a 3-qubit register)
qc.h(1)       # Put q1 in superposition -> toggles between _0_ and _1_
# q0 stays 0 (no gate)

qc.measure([0,1,2],[0,1,2])

print(qc.draw())

sim = AerSimulator()
counts = sim.run(qc, shots=4096).result().get_counts()
print("Superposition of 4 and 6 counts:", counts)  # ~50% '100', ~50% '110'


          ┌─┐      
q_0: ─────┤M├──────
     ┌───┐└╥┘┌─┐   
q_1: ┤ H ├─╫─┤M├───
     ├───┤ ║ └╥┘┌─┐
q_2: ┤ X ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2 
Superposition of 4 and 6 counts: {'110': 2090, '100': 2006}


Creating a quantum half adder

In [4]:
qc1 = QuantumCircuit(4,2) # 2 qubits and 2 classical bits
qc1.x(0) # apply NOT gate on qubit 0
qc1.x(1) # apply NOT gate on qubit 1
qc1.cx(0,2) # CNOT gate controlled by qubit 0 and targeting qubit 2
qc1.cx(1,2) # CNOT gate controlled by qubit 1 and targeting qubit 2
qc1.ccx(0,1,3)
qc1.measure(2,0)
qc1.measure(3,1)
display(qc1.draw()) 

job = sim.run(qc1)
result = job.result()
print("Results: ", result.get_counts())

┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

Results:  {'10': 1024}


Entangling States:

In [5]:
from qiskit.quantum_info import Statevector

qc2 = QuantumCircuit(2)
ket = Statevector(qc2)
ket.draw()

'Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],\n            dims=(2, 2))'

In [6]:
assert ket == Statevector.from_label('00')

In [7]:
qc2.cx(0,1)
ket = Statevector(qc2)
ket.draw()

'Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],\n            dims=(2, 2))'

In [16]:
# Grover's Algorithm (single marked item) — Qiskit mini demo
# Works for small n (e.g., 2–6 qubits). Requires qiskit and qiskit-aer.

from math import floor, pi, sqrt
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

# ---------- Utilities: oracle & diffuser ----------

def oracle_for(bitstring: str) -> QuantumCircuit:
    """
    Phase oracle that multiplies |bitstring> by -1 and leaves all other basis states unchanged.
    Implementation:
      1) X on qubits where target has '0' so that target maps to |11...1>
      2) n-controlled-Z (via H, MCX, H trick on last qubit)
      3) Uncompute the Xs
    """
    n = len(bitstring)
    qc = QuantumCircuit(n, name=f"Oracle({bitstring})")
    # Step 1: map target -> |11...1>
    for i, b in enumerate(bitstring[::-1]):   # little-endian vs visual: put LSB at qubit 0
        if b == '0':
            qc.x(i)
    # Step 2: n-controlled-Z using H-mcx-H on last qubit
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)  # multi-controlled X acts like controlled-Z in this sandwich
    qc.h(n-1)
    # Step 3: uncompute mapping
    for i, b in enumerate(bitstring[::-1]):
        if b == '0':
            qc.x(i)
    return qc

def diffuser(n: int) -> QuantumCircuit:
    """
    Standard Grover diffuser: 2|s><s| - I.
    Circuit: H^n → X^n → [H on last; MCX on last; H on last] → X^n → H^n
    """
    qc = QuantumCircuit(n, name="Diffuser")
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    return qc

# ---------- Build Grover circuit ----------
def grover_circuit(n_qubits: int, target_bitstring: str, iterations: int | None = None) -> QuantumCircuit:
    assert len(target_bitstring) == n_qubits, "Target bitstring must have length n_qubits"
    qc = QuantumCircuit(n_qubits, n_qubits)

    # Start state |0...0>, put into uniform superposition
    qc.h(range(n_qubits))

    # Pick iteration count if not provided
    if iterations is None:
        N = 2 ** n_qubits
        iterations = max(1, floor((pi/4) * sqrt(N)))

    O = oracle_for(target_bitstring)
    D = diffuser(n_qubits)

    for _ in range(iterations):
        qc.append(O.to_gate(), range(n_qubits))
        qc.append(D.to_gate(), range(n_qubits))

    # Measure all
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# ---------- Run a demo ----------
if __name__ == "__main__":
    # Choose a problem size and a target (write MSB...LSB; we map to little-endian internally)
    N_QUBITS = 3
    TARGET   = "111"   # mark |101>

    # Build circuit
    qc = grover_circuit(N_QUBITS, TARGET)

    # Simulate
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=2000).result()
    counts = result.get_counts()

    print("Measurement counts:", counts)

    # Optional: draw the circuit (text)
    print(qc.draw(fold=-1))

    # ---- Qiskit histogram (probabilities) ----
    # Some Qiskit versions do not support normalize=True.
    # So we convert counts -> probabilities ourselves and plot those.
    shots = sum(counts.values())
    probs = {k: v / shots for k, v in counts.items()}   # convert to probabilities

    fig = plot_histogram(
        probs,
        title="Grover Outcome Probabilities",
        sort='asc',
        bar_labels=False
    )

    # Make y-axis show probability (0..1); save always, show if GUI available.
    try:
        import matplotlib.pyplot as plt
        ax = fig.axes[0]
        ax.set_ylabel("Probability")
        ax.set_ylim(0, 1)
        fig.savefig("grover_hist.png", dpi=200, bbox_inches="tight")
        plt.show()
        print("Saved histogram to grover_hist.png")
    except Exception as e:
        # Fall back to saving without show (useful on headless systems)
        try:
            fig.savefig("grover_hist.png", dpi=200, bbox_inches="tight")
            print("Saved histogram to grover_hist.png (display skipped).")
        except Exception:
            print("(Histogram display and save skipped — matplotlib backend not available.)")


Measurement counts: {'001': 13, '100': 18, '101': 20, '000': 11, '010': 17, '110': 18, '011': 16, '111': 1887}
     ┌───┐┌──────────────┐┌───────────┐┌──────────────┐┌───────────┐┌─┐      
q_0: ┤ H ├┤0             ├┤0          ├┤0             ├┤0          ├┤M├──────
     ├───┤│              ││           ││              ││           │└╥┘┌─┐   
q_1: ┤ H ├┤1 Oracle(111) ├┤1 Diffuser ├┤1 Oracle(111) ├┤1 Diffuser ├─╫─┤M├───
     ├───┤│              ││           ││              ││           │ ║ └╥┘┌─┐
q_2: ┤ H ├┤2             ├┤2          ├┤2             ├┤2          ├─╫──╫─┤M├
     └───┘└──────────────┘└───────────┘└──────────────┘└───────────┘ ║  ║ └╥┘
c: 3/════════════════════════════════════════════════════════════════╩══╩══╩═
                                                                     0  1  2 
Saved histogram to grover_hist.png
